In [33]:
from scipy import stats
import plotly.express as px
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from sklearn.metrics import classification_report
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [34]:
df=pd.read_csv("/content/logatta.csv")
df.head()

,Age,BusinessTravel,Education,MaritalStatus,OverTime,EmployeeNumber,DailyRate,Gender,accepted for the interview
0,41,Travel_Rarely,2,Single,Yes,1,1102,Female,True
1,49,Travel_Frequently,1,Married,No,2,279,Male,False
2,37,Travel_Rarely,2,Single,Yes,4,1373,Male,False
3,33,Travel_Frequently,4,Married,Yes,5,1392,Female,False
4,27,Travel_Rarely,1,Married,No,7,591,Male,False


In [35]:
encoder=OrdinalEncoder()
for colums in df.columns:
  if df[colums].dtype=="object":
    df[colums]=encoder.fit_transform(df[[colums]])
df["accepted for the interview"]=encoder.fit_transform(df[["accepted for the interview"]])
df.head()

,Age,BusinessTravel,Education,MaritalStatus,OverTime,EmployeeNumber,DailyRate,Gender,accepted for the interview
0,41,2.0,2,2.0,1.0,1,1102,0.0,1.0
1,49,1.0,1,1.0,0.0,2,279,1.0,0.0
2,37,2.0,2,2.0,1.0,4,1373,1.0,0.0
3,33,1.0,4,1.0,1.0,5,1392,0.0,0.0
4,27,2.0,1,1.0,0.0,7,591,1.0,0.0


In [36]:
stdsacle=StandardScaler()
for colums in df.columns:
  if df[colums].dtype=="int64":
    df[colums]=stdsacle.fit_transform(df[[colums]])
df.head()

,Age,BusinessTravel,Education,MaritalStatus,OverTime,EmployeeNumber,DailyRate,Gender,accepted for the interview
0,0.452168,2.0,-0.897027,2.0,1.0,-1.710930,0.735781,0.0,1.0
1,1.331314,1.0,-1.876080,1.0,0.0,-1.709278,-1.303018,1.0,0.0
2,0.012595,2.0,-0.897027,2.0,1.0,-1.705975,1.407124,1.0,0.0
3,-0.426978,1.0,1.061078,1.0,1.0,-1.704323,1.454192,0.0,0.0
4,-1.086338,2.0,-1.876080,1.0,0.0,-1.701020,-0.530108,1.0,0.0


In [37]:
matrix = df.corr(numeric_only=True)
fig=px.imshow(matrix,x=matrix.columns,y=matrix.index,color_continuous_scale='RdBu_r',text_auto='.2f',aspect="auto")
fig.update_layout(title='Correlation Matrix',xaxis_showgrid=False,yaxis_showgrid=False,width=800,height=500,)
fig.show()

Compare to accepted for the interview , hightest corrolations have BiusinessTravell, Education and OverTime

In [38]:
x_train,x_test,y_train,y_test=train_test_split(list(zip(df["BusinessTravel"],df["OverTime"],df["Education"])),df["accepted for the interview"],test_size=0.2,random_state=42,shuffle=True)
dict_train={"BusinessTravel OverTime Education":x_train,"accepted for the interview":y_train,"data":"Training"}
dict_test={"BusinessTravel OverTime Education":x_test,"accepted for the interview":y_test,"data":"Test"}
df_all=pd.concat([pd.DataFrame(dict_train),pd.DataFrame(dict_test)],ignore_index=True)
df_all.head()

,BusinessTravel OverTime Education,accepted for the interview,data
0,"(2.0, 0.0, 1.0610777606082311)",0.0,Training
1,"(2.0, 0.0, 0.08202518248362939)",0.0,Training
2,"(2.0, 0.0, 0.08202518248362939)",0.0,Training
3,"(2.0, 0.0, -1.876079973765574)",0.0,Training
4,"(2.0, 0.0, 0.08202518248362939)",0.0,Training


Logistic regression

In [39]:
log_model=LogisticRegression()
log_model.fit(x_train,y_train)
accuracy = log_model.score(x_test, y_test)
print(f"Model Accuracy: {accuracy}")
predicted = log_model.predict([[33,0,7]])
print(f"Prediction for input: {predicted}")

Model Accuracy: 0.903010033444816
Prediction for input: [1.]


In [40]:
y_pred=log_model.predict(x_test)
confusion_matrix = metrics.confusion_matrix(y_test, y_pred)
print("Confusion matrix for LogisticRegression")
print(confusion_matrix)

Confusion matrix for LogisticRegression
[[260   8]
 [ 21  10]]


In [41]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

         0.0       0.93      0.97      0.95       268
         1.0       0.56      0.32      0.41        31

    accuracy                           0.90       299
   macro avg       0.74      0.65      0.68       299
weighted avg       0.89      0.90      0.89       299



KNN

In [42]:
knn_model=KNeighborsClassifier(n_neighbors=11)
knn_model.fit(x_train,y_train)
accuracy=knn_model.score(x_test,y_test)
print(f"Model Accuracy: {accuracy}")
predicted =knn_model.predict([[33,1,7]])
print(f"Prediction for input: {predicted}")

Model Accuracy: 0.8695652173913043
Prediction for input: [0.]


In [43]:
y_pred=knn_model.predict(x_test)
confusion_matrix = metrics.confusion_matrix(y_test, y_pred)
print("Confusion matrix for KNN")
print(confusion_matrix)

Confusion matrix for KNN
[[246  22]
 [ 17  14]]


In [44]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

         0.0       0.94      0.92      0.93       268
         1.0       0.39      0.45      0.42        31

    accuracy                           0.87       299
   macro avg       0.66      0.68      0.67       299
weighted avg       0.88      0.87      0.87       299



Gaussian Naive Bayes

In [45]:
nb_model= GaussianNB()
nb_model.fit(x_train,y_train)
accuracy=nb_model.score(x_test,y_test)
print(f"Model Accuracy: {accuracy}")
predicted =knn_model.predict([[33,0,77]])
print(f"Prediction for input: {predicted}")

Model Accuracy: 0.8595317725752508
Prediction for input: [0.]


In [46]:
y_pred=nb_model.predict(x_test)
confusion_matrix = metrics.confusion_matrix(y_test, y_pred)
print("Confusion matrix for Gaussian Naive Bayes")
print(confusion_matrix)

Confusion matrix for Gaussian Naive Bayes
[[229  39]
 [  3  28]]


In [47]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

         0.0       0.99      0.85      0.92       268
         1.0       0.42      0.90      0.57        31

    accuracy                           0.86       299
   macro avg       0.70      0.88      0.74       299
weighted avg       0.93      0.86      0.88       299



In [48]:
df=pd.read_csv("/content/logatta.csv")
fig=make_subplots(rows=1,cols=2,subplot_titles=("Gender","Age"),specs=[[{'type': 'domain'}, {'type': 'domain'}]])
fig.add_trace(go.Pie(labels=df["Gender"], values=None, hole=0.4, name="Gender"),row=1, col=1)
count,bin=np.histogram(df["Age"],bins=10)
label=[f"{int(bin[i])}-{int(bin[i+1])}" for i in range(len(count))]
new_df=pd.DataFrame({"Label":label,"Count":count})
fig.add_trace(go.Pie(labels=new_df["Label"], values=count, hole=0.4, name="Age"),row=1,col=2)
fig.show()